# データ分析入門編：scikit-learnで交差検証！
以前作成したBoston House-Priceデータセットで作成したSVRのモデルを交差検証を使ってチューニングします。
まずはデータをロードします。

In [1]:
import sklearn
from sklearn import datasets
boston = datasets.load_boston()

SVRは事前にデータを正規化する必要がありますので、StandardScalr()関数を使って正規化を行います。

In [2]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(boston.data)
data = scaler.transform(boston.data)

それでは、まずはチューニング前のデフォルトの設定の精度を見ましょう。精度の司法としては平均二乗誤差を使います。

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, boston.target, test_size=0.4, random_state=0)

In [4]:
from sklearn.svm import SVR
clf = SVR()
clf.fit(X_train, y_train) 
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, clf.predict(X_test))

34.165910124601005

結果、平均二乗誤差は34となりました。これが今回のベンチマークとなります。これよりも少ない誤差を目指しましょう。

次に、交差検証とグリッドサーチを使って最適なハイパーパラメーターを探します。今回対象とするハイパーパラメーターは"C"です。デフォルトの値は1.0ですが、それを変えてみましょう。

In [5]:
from sklearn.model_selection import GridSearchCV
parameters = {'C':[0.001, 0.01, 0.1, 1.0, 10, 100, 1000, 10000]}
clf = GridSearchCV(SVR(), parameters, cv=10, scoring='neg_mean_squared_error')
clf.fit(data, boston.target)

GridSearchCV(cv=10, error_score='raise',
       estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.001, 0.01, 0.1, 1.0, 10, 100, 1000, 10000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

一番良かったパラメーターはclf.best_estimator_で確認できます。

In [6]:
clf.best_estimator_

SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

上記の通り、一番よいCの値は100となりました。では、実際にC=100と設定して、精度がどれほど変わるか見てみましょう。

In [7]:
from sklearn.svm import SVR
clf = SVR(C=100)
clf.fit(X_train, y_train) 
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, clf.predict(X_test))

14.384304813554243

平均二乗誤差が34から14に減り、大分精度があがったことが分かります。